In [1]:
import os
import re
from langchain_openai import ChatOpenAI

In [41]:
with open('.env') as f:
    for line in f.read().splitlines():
        if line.startswith("#"):
            continue
        k, v = line.split('=')
        os.environ[k] = v

In [42]:
oai_llm = ChatOpenAI(api_key=os.environ["OAI_KEY_1"], model="gpt-4", base_url=os.environ.get("OAI_BASE_URL", None))

In [43]:
system_prompt = "You are a helpful storywriting assistant."

In [44]:
chapter = open("out/chapter_5.txt").read()

Identify characters in the chapter for use when marking which characters turn it is

In [48]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"""Give the following story, please point out each named character present in the Scene. List only the characters in a list like the following, include any other way they are referred to in the story
1. Character 1, Other name used in chapter
2. Character 2

{chapter}"""}
]

characters = oai_llm.invoke(messages).content
print(characters)

1. Erik Maria Bark, Erik
2. Simone, Sixan
3. Joona Linna, the detective
4. Daniella Richards, the doctor in charge
5. Erik's son, not named
6. The fifteen-year-old boy, the patient, the boy
7. Magnus, Daniella's husband (mentioned but not physically present)
8. The blond man, the lanky fellow
9. Two uniformed cops (not individually named)


In [46]:
# given an index i get the ith line and the surrounding 3 lines before and after
def get_lines(lines, i, context=3):
  start = max(0, i - context)
  end = min(len(lines), i + context + 1)
  return "\n".join(lines[start:end])

Here we go over each line of the chapter adding character markers to indicate the turn of the actions and speech. We add surrounding lines and the list of characters as context.

In [56]:
lines = chapter.splitlines()
cost = 0
output = ""
for i, line in enumerate(lines):
    messages = [
        { "role": "system", "content": system_prompt },
        { "role": "user", "content": f"""Please insert markers for when the character acting changes. You should only add markers to the single linge given.
Here are the surroinding lines that will help:
The detective rapidly summarizes the patient's status, concluding, "He hasn't been stabilized. He's in circulatory shock and unconscious."
"Who's the doctor in charge?" asks Erik.
"Daniella Richards."
"She's extremely capable. I'm sure she can—"
"She was the one who asked me to call you. She needs your help. It's urgent."
When Erik returns to the bedroom to get his clothes, Simone is lying on her back, looking at him with a strange, empty expression. A strip of light from the streetlamp is shining in between the blinds.
"I didn't mean to wake you," he says softly.
         
Here are the characters in the schene:
1. Erik Maria Bark, Erik
2. Simone, Sixan
3. Joona Linna, the detective
4. Daniella Richards, the doctor in charge
5. Erik's son, not named
6. The fifteen-year-old boy, the patient, the boy
7. Magnus, Daniella's husband (mentioned but not physically present)
8. The blond man, the lanky fellow
9. Two uniformed cops (not individually named)

Here is the line that you must add the markers to:
"Daniella Richards."
The markers should only indicate indicate the character which is acting, and should ignore if another character is mentioned in the line but is not acting any actions."""},
        { "role": "assistant", "content": """Joona Linna: "Daniella Richards.\""""},
        { "role": "user", "content": f"""Please insert markers for when the character acting changes. You should only add markers to the single linge given.
Here are the surroinding lines that will help:
The detective rapidly summarizes the patient's status, concluding, "He hasn't been stabilized. He's in circulatory shock and unconscious."
"Who's the doctor in charge?" asks Erik.
"Daniella Richards."
"She's extremely capable. I'm sure she can—"
"She was the one who asked me to call you. She needs your help. It's urgent."
When Erik returns to the bedroom to get his clothes, Simone is lying on her back, looking at him with a strange, empty expression. A strip of light from the streetlamp is shining in between the blinds.
"I didn't mean to wake you," he says softly.
         
Here are the characters in the schene:
1. Erik Maria Bark, Erik
2. Simone, Sixan
3. Joona Linna, the detective
4. Daniella Richards, the doctor in charge
5. Erik's son, not named
6. The fifteen-year-old boy, the patient, the boy
7. Magnus, Daniella's husband (mentioned but not physically present)
8. The blond man, the lanky fellow
9. Two uniformed cops (not individually named)

Here is the line that you must add the markers to:
"She's extremely capable. I'm sure she can—"
The markers should only indicate indicate the character which is acting, and should ignore if another character is mentioned in the line but is not acting any actions."""},
        { "role": "assistant", "content": """Erik Maria Bark: "She's extremely capable. I'm sure she can—\""""},
        { "role": "user", "content": f"""Please insert markers for when the character acting changes. You should only add markers to the single linge given.
Here are the surroinding lines that will help:
"Daniella Richards."
"She's extremely capable. I'm sure she can—"
"She was the one who asked me to call you. She needs your help. It's urgent."
When Erik returns to the bedroom to get his clothes, Simone is lying on her back, looking at him with a strange, empty expression. A strip of light from the streetlamp is shining in between the blinds.
"I didn't mean to wake you," he says softly.
"Who was that?" she asks.
"Police... a detective... I didn't catch his name."
         
Here are the characters in the schene:
1. Erik Maria Bark, Erik
2. Simone, Sixan
3. Joona Linna, the detective
4. Daniella Richards, the doctor in charge
5. Erik's son, not named
6. The fifteen-year-old boy, the patient, the boy
7. Magnus, Daniella's husband (mentioned but not physically present)
8. The blond man, the lanky fellow
9. Two uniformed cops (not individually named)

Here is the line that you must add the markers to:
When Erik returns to the bedroom to get his clothes, Simone is lying on her back, looking at him with a strange, empty expression. A strip of light from the streetlamp is shining in between the blinds.
The markers should only indicate indicate the character which is acting, and should ignore if another character is mentioned in the line but is not acting any actions."""},
        { "role": "assistant", "content": """Erik Maria Bark: When Erik returns to the bedroom to get his clothes, 
Simone: Simone is lying on her back, looking at him with a strange, empty expression. A strip of light from the streetlamp is shining in between the blinds."""},
        { "role": "user", "content": f"""Please insert markers for when the character acting changes. You should only add markers to the single linge given.
Here are the surroinding lines that will help:
{get_lines(lines, i)}        
         
Here are the characters in the schene:
{", ".join(characters)}

Here is the line that you must add the markers to:
{line}
The markers should only indicate indicate the character which is acting, and should ignore if another character is mentioned in the line but is not acting any actions."""},
    ]
    
    res = oai_llm.invoke(
        messages,
    )
    mes = res.content
    usage = res.response_metadata["token_usage"]
    cost += (usage["completion_tokens"] * 0.06 + usage["prompt_tokens"] * 0.03) / 1000
    print(mes)
    output += mes + "\n"
print(cost)

Erik Maria Bark: Erik Maria Bark is yanked reluctantly from his dream when the telephone rings. Before he is fully awake, he hears himself say with a smile, "Balloons and streamers."
Erik Maria Bark: His heart is pounding from the sudden awakening. Erik has no idea what he meant by these words. The dream is completely gone, as if he had never had it.
Erik Maria Bark: He fumbles to find the ringing phone, creeping out of the bedroom with it and closing the door behind him to avoid waking Simone. 
Joona Linna: A detective named Joona Linna asks if he is sufficiently awake to absorb important information. 
Erik Maria Bark: His thoughts are still tumbling down into the dark empty space after his dream as he listens.
Joona Linna: "I've heard you're very skilled in the treatment of acute trauma," says Linna.
Erik Maria Bark: "Yes," says Erik.
Erik Maria Bark: He swallows a painkiller as he listens. 
Joona Linna: The detective explains that he needs to question a fifteen-year-old boy who has 

The precomputed text with the turns serves as a "demo checkpoint" so that we avoid needing to rerun the marker portion again.

In [10]:
# output = """<Erik>Erik Maria Bark is yanked reluctantly from his dream when the telephone rings. Before he is fully awake, he hears himself say with a smile, "Balloons and streamers."
# <Erik>His heart is pounding from the sudden awakening. Erik has no idea what he meant by these words. The dream is completely gone, as if he had never had it.
# <Erik>He fumbles to find the ringing phone, creeping out of the bedroom with it and closing the door behind him to avoid waking Simone. <Joona>A detective named Joona Linna asks if he is sufficiently awake to absorb important information. <Erik>His thoughts are still tumbling down into the dark empty space after his dream as he listens.
# <Joona Linna>"I've heard you're very skilled in the treatment of acute trauma," says Linna.
# <Erik>"Yes," says Erik.
# <Erik>He swallows a painkiller as he listens. <Joona Linna>The detective explains that he needs to question a fifteen-year-old boy who has witnessed a double murder and been seriously injured himself. During the night he was moved from the neurological unit in Huddinge to the neurosurgical unit at Karolinska University Hospital in Solna.
# <Erik>"What's his condition?" Erik asks.
# <Joona Linna>The detective rapidly summarizes the patient's status, concluding, "He hasn't been stabilized. He's in circulatory shock and unconscious."
# <Erik>"Who's the doctor in charge?" asks Erik.
# <Joona Linna>"Daniella Richards."
# <Erik>"She's extremely capable. I'm sure she can—"
# <Joona Linna>"She was the one who asked me to call you. She needs your help. It's urgent."
# <Erik>When Erik returns to the bedroom to get his clothes, <Simone>Simone is lying on her back, looking at him with a strange, empty expression. A strip of light from the streetlamp is shining in between the blinds.
# <Erik>"I didn't mean to wake you," he says softly.
# <Simone>"Who was that?" she asks.
# <Erik>"Police... a detective... I didn't catch his name."
# <Simone>"What's it about?"
# <Erik>"I have to go to the hospital," he replies. "They need some help with a boy.\""""

In [71]:
# we wanna go over and interrogate the AI on whether or not the speaker is correctly identified given surrounding context
# this should hopefully improve the results as the context wont be filled with as much stuff
lines = output.splitlines()
conversation = []
for i, line in enumerate(lines):
    messages = [
        { "role": "system", "content": system_prompt },
        { "role": "user", "content": f"""Given the following context please check and correct the given lines speaker.
Only respond with the correct speaker.
The lines will be in the format of:
```
Speaker: line
```
         
The characters that are present in this chapter are:
1. Erik Maria Bark, Erik
2. Simone, Sixan
3. Joona Linna, the detective
4. Daniella Richards, the doctor in charge
5. Erik's son, not named
6. The fifteen-year-old boy, the patient, the boy
7. Magnus, Daniella's husband (mentioned but not physically present)
8. The blond man, the lanky fellow
9. Two uniformed cops (not individually named)

Here is the surrounding lines to help clarify:
Daniella Richards: "Have some coffee," she says.
Erik Maria Bark: "Do we have time?" asks Erik.
Daniella Richards: "I've got the bleed in the liver under control," she replies.
The Blond Man: A man of about forty-five, dressed in jeans and a black jacket, is thumping the coffee machine. He has tousled blond hair, and his lips are serious, clamped firmly together. 
Erik Maria Bark: Erik thinks maybe this is Daniella's husband, Magnus. He has never met him; he has only seen a photograph in her office.

Here is the line to identify:
Daniella Richards: "I've got the bleed in the liver under control," she replies."""},
        { "role": "assistant", "content": "Daniella Richards" },
        { "role": "user", "content": f"""Given the following context please check and correct the given lines speaker.
Only respond with the correct speaker.
The lines will be in the format of:
```
Speaker: line
```
         
The characters that are present in this chapter are:
1. Erik Maria Bark, Erik
2. Simone, Sixan
3. Joona Linna, the detective
4. Daniella Richards, the doctor in charge
5. Erik's son, not named
6. The fifteen-year-old boy, the patient, the boy
7. Magnus, Daniella's husband (mentioned but not physically present)
8. The blond man, the lanky fellow
9. Two uniformed cops (not individually named)
        
Here is the surrounding lines to help clarify:
Joona Linna: The detective rapidly summarizes the patient's status, concluding, "He hasn't been stabilized. He's in circulatory shock and unconscious."
Erik Maria Bark: "Who's the doctor in charge?" asks Erik.
Erik Maria Bark: "Daniella Richards."
Erik Maria Bark: "She's extremely capable. I'm sure she can—"
Joona Linna: "She was the one who asked me to call you. She needs your help. It's urgent."

Here is the line to identify
Erik Maria Bark: "Daniella Richards.\""""},
        { "role": "assistant", "content": "Joona Linna\nThe text is of a conversation between Erik and Joona, and as such should go in turns." },
        { "role": "user", "content": f"""Given the following context please check and correct the given lines speaker.
Only respond with the correct speaker. If none of the characters in the list match, please respond with the orignal speaker in the line.
The lines will be in the format of:
```
Speaker: line
```
         
The characters that are present in this chapter are:
{characters}

Here is the surrounding lines to help clarify:
{get_lines(output, i)}

Here is the line to identify
{line}
"""},
    ]

    response = oai_llm.invoke(messages).content
    speaker = response.split(": ")[0]
    orig_speaker = line.split(": ")[0]
    message = line.split(": ", maxsplit=1)[1]
    conversation.append({
        "from": orig_speaker if "narrator" in speaker.lower() else speaker,
        "value": message,
    })


In [73]:
for line, conv in zip(lines, conversation):
    if "narrator" in conv["from"].lower():
        print(conv)
        print(line)
        conv["from"] = line.split(": ")[0]
conversation

{'from': 'Narrator', 'value': 'It\'s three o\'clock in the morning when Erik gets into his car. Snow falls slowly from the black sky. There is not a breath of wind, and the heavy flakes settle sleepily on the empty street. He turns the key in the ignition, and the music pours in like a soft wave: Miles Davis, "Kind of Blue."'}
Erik Maria Bark: It's three o'clock in the morning when Erik gets into his car. Snow falls slowly from the black sky. There is not a breath of wind, and the heavy flakes settle sleepily on the empty street. He turns the key in the ignition, and the music pours in like a soft wave: Miles Davis, "Kind of Blue."
{'from': 'The Narrator', 'value': 'He inserts his pass card, keys in the six-digit code, enters the lobby, takes the lift to the fifth floor, and walks down the hall. The blue vinyl floors shine like ice, and the corridor smells of antiseptic. Only now does he become aware of his fatigue, following the sudden surge of adrenaline brought on by the call. It ha

[{'from': 'Erik Maria Bark',
  'value': 'Erik Maria Bark is yanked reluctantly from his dream when the telephone rings. Before he is fully awake, he hears himself say with a smile, "Balloons and streamers."'},
 {'from': 'Erik Maria Bark',
  'value': 'His heart is pounding from the sudden awakening. Erik has no idea what he meant by these words. The dream is completely gone, as if he had never had it.'},
 {'from': 'Erik Maria Bark',
  'value': 'He fumbles to find the ringing phone, creeping out of the bedroom with it and closing the door behind him to avoid waking Simone. '},
 {'from': 'Joona Linna',
  'value': 'A detective named Joona Linna asks if he is sufficiently awake to absorb important information. '},
 {'from': 'Erik Maria Bark',
  'value': 'His thoughts are still tumbling down into the dark empty space after his dream as he listens.'},
 {'from': 'Joona Linna',
  'value': '"I\'ve heard you\'re very skilled in the treatment of acute trauma," says Linna.'},
 {'from': 'Erik Maria 

In [83]:
def split_quoted_and_non_quoted(s):
    # Regular expression to match quoted and non-quoted parts
    pattern = re.compile(r'"([^"]+)"|([^"]+)')
    
    # Find all matches
    matches = pattern.finditer(s)
    
    # Process matches to form the result list
    result = []
    for match in matches:
        if match.group(1):
            result.append({"value": match.group(1).strip(), "type": "quoted"})
        elif match.group(2):
            # Further split non-quoted parts by spaces while preserving spaces
            result.append({"value": match.group(2).strip(), "type": "unquoted"})
    
    return result

clarify character incase of slight hallucinations mispelling name or leaving out some part of it

In [76]:
for message in conversation:
        messages = [
                { "role": "system", "content": system_prompt },
                { "role": "user", "content": f"""Given the following list of characters:
1. Erik Maria Bark, Erik
2. Simone, Sixan
3. Joona Linna, the detective
4. Daniella Richards, the doctor in charge
5. Erik's son, not named
6. The fifteen-year-old boy, the patient, the boy
7. Magnus, Daniella's husband (mentioned but not physically present)
8. The blond man, the lanky fellow
9. Two uniformed cops (not individually named)

The character list is given in the following format
1. Name, alias
2. Name, alias

Which character is the following referring to, use only the characters as in the list and provide only their name.
Joona"""},
                { "role": "assistant", "content": "Joona Linna" },
                { "role": "user", "content": f"""Given the following list of characters:
1. Erik Maria Bark, Erik
2. Simone, Sixan
3. Joona Linna, the detective
4. Daniella Richards, the doctor in charge
5. Erik's son, not named
6. The fifteen-year-old boy, the patient, the boy
7. Magnus, Daniella's husband (mentioned but not physically present)
8. The blond man, the lanky fellow
9. Two uniformed cops (not individually named)

The character list is given in the following format
1. Name, alias
2. Name, alias

Which character is the following referring to, use only the characters as in the list.
Maria Bark"""},
                { "role": "assistant", "content": "Erik Maria Bark" },
                { "role": "user", "content": f"""Given the following list of characters:
{characters}

Which character is the following referring to, use only the characters as in the list.
{message["from"]}"""},
        ]
        output = oai_llm.invoke(messages, stop=["\n"])
        print(f"{message['from']}: {output.content}")
        message["from"] = re.sub(r",.*", "", re.sub(r"\(.+?\)", "", output.content)).strip()

Erik Maria Bark: Erik Maria Bark
Erik Maria Bark: Erik Maria Bark
Erik Maria Bark: Erik Maria Bark
Joona Linna: Joona Linna
Erik Maria Bark: Erik Maria Bark
Joona Linna: Joona Linna
Erik Maria Bark: Erik Maria Bark
Erik Maria Bark: Erik Maria Bark
Joona Linna: Joona Linna
Erik Maria Bark: Erik Maria Bark
Joona Linna: Joona Linna
Erik Maria Bark: Erik Maria Bark
Joona Linna: Joona Linna
Erik Maria Bark: Erik Maria Bark
Joona Linna: Joona Linna
Erik Maria Bark: Erik Maria Bark
Simone: Simone
Erik Maria Bark: Erik Maria Bark
Simone: Simone
Erik Maria Bark: Erik Maria Bark
Simone: Simone
Erik Maria Bark: Erik Maria Bark
Simone: Simone
Erik Maria Bark: Erik Maria Bark
Erik Maria Bark: Erik Maria Bark
Erik Maria Bark: Erik Maria Bark
Erik Maria Bark: Erik Maria Bark
Erik Maria Bark: Erik Maria Bark
Erik Maria Bark: Erik Maria Bark
Erik Maria Bark: Erik Maria Bark
Erik Maria Bark: Erik Maria Bark
Erik Maria Bark: Erik Maria Bark
Two uniformed cops: Two uniformed cops
Erik Maria Bark: Erik Mar

Here we rewrite the non-spoken dialogue into first person e.g. Erik does xyz -> I do xyz  
We give it the full context of the character as was originally pointed out in the initial prompt to hopefully get things such as nicknames or similar in the context as well.

In [84]:
def rewrite_perspective(char, message):
    chars = [re.sub(r"^\d+. ", "", character) for character in characters.splitlines()]
    char = [c for c in chars if char in c][0]

    messages = [
        { "role": "system", "content": system_prompt },
        { "role": "user", "content": f"""Please rewrite the following line to be from a first person present tense perspective. The character acting out the following line is Erik
Avoid using the characters name in the rewritten text, keep in mind that pronouns may refer to other characters and should as such stay the same.
Do not clarify who {char} is, like with "I, {char}" or "I, occupation"

Here is the line to rewrite:
says Erik""" },
        { "role": "assistant", "content": "I say" },
        { "role": "user", "content": f"""Please rewrite the following line to be from a first person present tense perspective. The character acting out the following line is Erik
Avoid using the characters name in the rewritten text, keep in mind that pronouns may refer to other characters and should as such stay the same.
Do not clarify who {char} is, like with "I, {char}" or "I, occupation"

Here is the line to rewrite:
When Erik returns to the bedroom to get his clothes,""" },
        { "role": "assistant", "content": "When I return to the bedroom to get my clothes," },
        { "role": "user", "content": f"""Please rewrite the following line to be from a first person present tense perspective. The character acting out the following line is {char}
Avoid using the characters name in the rewritten text, keep in mind that pronouns may refer to other characters and should as such stay the same.
Do not clarify who {char} is, like with "I, {char}" or "I, occupation"
         
Here is the line to rewrite:
{message}""" },
    ]

    return oai_llm.invoke(messages).content

results = []
for message in conversation:
    # remove speech from message as that shouldn't change perspective
    split_message = split_quoted_and_non_quoted(message["value"])
    # then call rewrite_perspective on each part of remaining text individually and join them back with the speech in the correct position
    rewritten = []
    for part in split_message:
        if part["type"] == "unquoted":
            content = rewrite_perspective(message["from"], part["value"])
        else:
            content = f'"{part["value"]}"'
        rewritten.append(content)
    print(f"{message['from']}: {' '.join(rewritten)}")
    results.append({ "from": message["from"], "value": " ".join(rewritten) })

Erik Maria Bark: I'm yanked reluctantly from my dream when the telephone rings. Before I'm fully awake, I hear myself say with a smile, "Balloons and streamers."
Erik Maria Bark: My heart is pounding from the sudden awakening. I have no idea what I meant by these words. The dream is completely gone, as if I had never had it.
Erik Maria Bark: I fumble to find the ringing phone, creeping out of the bedroom with it and closing the door behind me to avoid waking Simone.
Joona Linna: I ask if he is sufficiently awake to absorb important information.
Erik Maria Bark: My thoughts are still tumbling down into the dark empty space after my dream as I listen.
Joona Linna: "I've heard you're very skilled in the treatment of acute trauma," I say.
Erik Maria Bark: "Yes," I say.
Erik Maria Bark: I swallow a painkiller as I listen.
Joona Linna: I explain that I need to question a fifteen-year-old boy who has witnessed a double murder and been seriously injured himself. During the night, he was moved 

Concatenate consecutive messags from the same character

In [85]:
current_char = None
new_results = []
current_mes = ""
for mes in results:
    if mes['from'] != current_char:
        if current_char:
            new_results.append({ "from": current_char, "value": current_mes})
        current_mes = mes['value']
        current_char = mes['from']
    else:
        current_mes += f" {mes['value']}"

new_results.append({ "from": current_char, "value": current_mes})
print(new_results)

[{'from': 'Erik Maria Bark', 'value': 'I\'m yanked reluctantly from my dream when the telephone rings. Before I\'m fully awake, I hear myself say with a smile, "Balloons and streamers." My heart is pounding from the sudden awakening. I have no idea what I meant by these words. The dream is completely gone, as if I had never had it. I fumble to find the ringing phone, creeping out of the bedroom with it and closing the door behind me to avoid waking Simone.'}, {'from': 'Joona Linna', 'value': 'I ask if he is sufficiently awake to absorb important information.'}, {'from': 'Erik Maria Bark', 'value': 'My thoughts are still tumbling down into the dark empty space after my dream as I listen.'}, {'from': 'Joona Linna', 'value': '"I\'ve heard you\'re very skilled in the treatment of acute trauma," I say.'}, {'from': 'Erik Maria Bark', 'value': '"Yes," I say. I swallow a painkiller as I listen.'}, {'from': 'Joona Linna', 'value': 'I explain that I need to question a fifteen-year-old boy who ha

format into ChatML for visualization purposes

In [86]:
# format into chatml
def turn(char, message):
    return f"""<|im_start|>{char}
{message}<|im_end|>"""

print("\n".join([turn(result["from"], result["value"]) for result in new_results]))

<|im_start|>Erik Maria Bark
I'm yanked reluctantly from my dream when the telephone rings. Before I'm fully awake, I hear myself say with a smile, "Balloons and streamers." My heart is pounding from the sudden awakening. I have no idea what I meant by these words. The dream is completely gone, as if I had never had it. I fumble to find the ringing phone, creeping out of the bedroom with it and closing the door behind me to avoid waking Simone.<|im_end|>
<|im_start|>Joona Linna
I ask if he is sufficiently awake to absorb important information.<|im_end|>
<|im_start|>Erik Maria Bark
My thoughts are still tumbling down into the dark empty space after my dream as I listen.<|im_end|>
<|im_start|>Joona Linna
"I've heard you're very skilled in the treatment of acute trauma," I say.<|im_end|>
<|im_start|>Erik Maria Bark
"Yes," I say. I swallow a painkiller as I listen.<|im_end|>
<|im_start|>Joona Linna
I explain that I need to question a fifteen-year-old boy who has witnessed a double murder an

In [87]:
import json
with open("larskepler_thehypnotist.jsonl", "w") as f:
    for message in conversation:
        f.write(json.dumps(message) + "\n")